<a href="https://colab.research.google.com/github/wesslen/llm-examples/blob/main/notebooks/dspy_gsm8k_save_bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!uv pip install --system dspy
!uv pip install --system litellm[proxy]

Using Python 3.11.13 environment at: /usr
Resolved 79 packages in 1.28s
⠙ Preparing packages... (0/13)
⠙ Preparing packages... (0/13)
python-dotenv ------------------------------     0 B/20.07 KiB
⠙ Preparing packages... (0/13)
backoff    ------------------------------     0 B/14.79 KiB
python-dotenv ------------------------------     0 B/20.07 KiB
⠙ Preparing packages... (0/13)
colorlog   ------------------------------     0 B/11.16 KiB
backoff    ------------------------------     0 B/14.79 KiB
python-dotenv ------------------------------     0 B/20.07 KiB
⠙ Preparing packages... (0/13)
colorlog   ------------------------------     0 B/11.16 KiB
backoff    ------------------------------     0 B/14.79 KiB
python-dotenv ------------------------------     0 B/20.07 KiB
⠙ Preparing packages... (0/13)
colorlog   ------------------------------     0 B/11.16 KiB
backoff    ------------------------------     0 B/14.79 KiB
python-dotenv ------------------------------ 14.88 KiB/20.07 KiB
⠙ Pre

In [3]:
import dspy
from google.colab import userdata

lm = dspy.LM("gemini/gemini-2.5-flash", api_key=userdata.get('GOOGLE_API_KEY'))
dspy.configure(lm=lm)

In [5]:
lm("Hello")

['Hello there! How can I help you today?']

In [8]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

dspy.settings.configure(lm=lm)

gsm8k = GSM8K()
gsm8k_trainset = gsm8k.train[:10]
dspy_program = dspy.ChainOfThought("question -> answer")

optimizer = dspy.BootstrapFewShot(metric=gsm8k_metric, max_bootstrapped_demos=4, max_labeled_demos=4, max_rounds=5)
compiled_dspy_program = optimizer.compile(dspy_program, trainset=gsm8k_trainset)

 40%|████      | 4/10 [00:00<00:00, 529.58it/s]

Bootstrapped 4 full traces after 4 examples for up to 5 rounds, amounting to 4 attempts.


In [32]:
dspy_program(question="How many R's are in strawberrry?")

Prediction(
    reasoning='To find the number of \'R\'s in "strawberrry", I will count each instance of the letter \'r\' (case-insensitive) in the given word.\nThe word is "strawberrry".\n1. s\n2. t\n3. r (1st R)\n4. a\n5. w\n6. b\n7. e\n8. r (2nd R)\n9. r (3rd R)\n10. r (4th R)\n11. y\nCounting them, there are 4 \'R\'s.',
    answer="There are 4 R's in strawberrry."
)

In [33]:
compiled_dspy_program(question="How many R's are in strawberrry?")

Prediction(
    reasoning='To find out how many R\'s are in "strawberrry", I will count each instance of the letter \'R\' in the given word.\ns-t-r-a-w-b-e-r-r-r-y\n1st R: s-t-**r**-a-w-b-e-r-r-r-y\n2nd R: s-t-r-a-w-b-e-**r**-r-r-y\n3rd R: s-t-r-a-w-b-e-r-**r**-r-y\n4th R: s-t-r-a-w-b-e-r-r-**r**-y\nThere are 4 \'R\'s in the word "strawberrry".',
    answer='4'
)

In [14]:
# dspy 2.6.0+, saves as program.pkl file
compiled_dspy_program.save("./dspy_program/", save_program=True)